In [1]:
import cv2
import mediapipe as mp
import os
import glob
import pandas as pd
import numpy as np

In [2]:
mp_drawing = mp.solutions.drawing_utils #점으로 손가락 마디 표시
mp_hands = mp.solutions.hands # 선으로 손가락 마디 표시
mp_drawing_styles = mp.solutions.drawing_styles

In [3]:
# 빈 dataframe 생성
df = pd.DataFrame()

In [4]:
x_list = []
y_list = []

In [27]:
video_file = './data/KakaoTalk_20230831_142745190.mp4'

cap = cv2.VideoCapture(video_file)

frame_num_list = []

with mp_hands.Hands() as hands:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # 프레임 번호를 추가합니다.
        frame_num = cap.get(cv2.CAP_PROP_POS_FRAMES)
        frame_num_list.append(frame_num)

        # MediaPipe를 사용하여 손을 감지합니다.
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        # 손의 랜드마크 x, y 좌표를 뽑습니다.
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i, lm in enumerate(hand_landmarks.landmark):
                    h,w,c = frame.shape
                    cx, cy = int(lm.x *w), int(lm.y*h)
                    print(i, ":", cx, cy)
                break
                    '''x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_list.append(x)
                    y_list.append(y)'''
    # 프레임별로 x, y 좌표를 DataFrame으로 변환합니다.
    df = pd.DataFrame({'frame_num': frame_num_list, 'x': x_list, 'y': y_list})

    print(df)

cap.release()


IndentationError: unexpected indent (4058207294.py, line 30)

### mediapipe로 프레임별로 이미지 캡쳐

In [41]:
# 동영상 파일의 경로를 리스트로 가져옴
file_list = glob.glob('./data/*.mp4')

# 각 동영상 파일에 대해 처리
for i, file in enumerate(file_list):
    cap = cv2.VideoCapture(file)
    file_name = os.path.splitext(os.path.basename(file))[0]  # 파일 이름 (확장자 제외)

    # 동영상 정보 가져오기
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # FPS : 1초에 몇 프레임이 들어가는가
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # 프레임 너비
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 프레임 높이
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # 프레임 수

    # 출력 폴더 생성
    output_dir = f'./data/output_{i}'
    os.makedirs(output_dir, exist_ok=True)

    with mp_hands.Hands(min_detection_confidence=0.5, #최소 탐지 신뢰도. 탐지가 성공한 것으로 간주되는 손 감지 모델의 최소 신뢰 값([0.0, 1.0]). 기본값은 0.5
                        min_tracking_confidence=0.5) as hands: #최소 추적 신뢰도
        
        count = 0
        
        while cap.isOpened():
            # ret : frame capture결과(boolean)
            # frame : Capture한 frame
            ret, frame = cap.read()

            if not ret:
                break

            # 손 감지
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            
            #x = []
            
            '''# k = landmarks 개수 및 번호
            # results.hand_landmarks.landmark[k].x/y/z/visibility로 
            # k번째 landmarks의 정보를 가져올 수 있다.
            for k in range(21):
                x.append(results.hand_landmarks.landmark[k].x)
                x.append(results.hand_landmarks.landmark[k].y)


            # list x를 dataframe으로 변경
            tmp = pd.DataFrame(x).T

            # dataframe에 정보 쌓아주기
            # 33개 landmarks의 132개 정보가 dataframe에 담긴다.
            df = pd.concat([df, tmp])'''
                    
            # 이미지에 손 주석을 그립니다.
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(), # 포인트
                        mp_drawing_styles.get_default_hand_connections_style()) # 선
                    for landmark in hand_landmarks.landmark:                            
                        x_list.append(landmark.x)
                        y_list.append(landmark.y)

            x_array = np.array(x_list)
            y_array = np.array(y_list)

            # 랜드마크의 x, y 좌표를 DataFrame으로 변환
            df = pd.DataFrame({'x': x_array, 'y': y_array})

            # 이미지 저장
            frame_filename = os.path.join(output_dir, f'frame_{count}.jpg')
            cv2.imwrite(frame_filename, frame)

            count += 1

    cap.release()



In [42]:
df

,x,y
0,0.292408,0.745893
1,0.408302,0.781159
2,0.517295,0.773079
3,0.611248,0.738449
4,0.699165,0.740333
5,0.582716,0.730566
6,0.690999,0.696964
7,0.757298,0.649591
8,0.808360,0.606800
9,0.553975,0.672205


In [12]:
# .mp4파일 가져오기 및 파일 별 정보
import glob
file_list = glob.glob('./data/*.mp4')
print(file_list)

# input
'''path = './data/'
file_list = os.listdir(path) #위 폴더에 있는 파일 출력
print(file_list)'''

for file in file_list:
    cap = cv2.VideoCapture(file) 

    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # 총 프레임 갯수
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) #fps : 1초당 몇장 보이는가

    print("length :", length)
    print("width :", width)
    print("height :", height)
    print("fps :", fps)
    print()

['./data\\KakaoTalk_20230831_142745190.mp4', './data\\KakaoTalk_20230831_142748198.mp4', './data\\output1.mp4', './data\\output2.mp4']
length : 226
width : 640
height : 480
fps : 24.0

length : 226
width : 640
height : 480
fps : 24.0

length : 226
width : 640
height : 480
fps : 24.0

length : 226
width : 640
height : 480
fps : 24.0



In [19]:
file_list[0][7:-4]

'KakaoTalk_20230831_142745190'

In [ ]:
# 저장 경로 설정
# output : 캡처이미지
'''output_video_path = './data/*.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))'''
# 폴더 생성
file_num = len(file_list)
for i in range(file_num):
   dirs = os.path.join('./data/', f'output_{i}')
   
   
'''try:
    if not os.path.exists(dirs):
        os.makedirs(dirs)
except OSError:
    print ('Error: Creating directory. ' +  file_list[7:-4])'''


count = 0

with mp_hands.Hands(
    model_complexity=0, #모델 복잡성은 0 또는 1. 랜드마크 정확도와 추론 지연 시간은 모델 복잡성과 함께 증가. 기본값은 1
    min_detection_confidence=0.5, #최소 탐지 신뢰도. 탐지가 성공한 것으로 간주되는 손 감지 모델의 최소 신뢰 값([0.0, 1.0]). 기본값은 0.5
    min_tracking_confidence=0.5) as hands: #최소 추적 신뢰도
  
  while cap.isOpened():
    # ret : frame capture결과(boolean)
    # image : Capture한 frame
    ret, image = cap.read()
    
    # 웹캠 사용할 경우
    '''if not ret:
      print("카메라를 찾을 수 없습니다.")
      # 동영상을 불러올 경우는 'continue' 대신 'break'를 사용합니다.
      break'''

    # 필요에 따라 성능 향상을 위해 이미지 작성을 불가능함으로 기본 설정합니다.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # 이미지에 손 주석을 그립니다.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(), # 포인트
            mp_drawing_styles.get_default_hand_connections_style()) # 선
        
    if not ret:
        break
    
    elif (int(cap.get(1)) % fps == 0): #앞서 불러온 fps 값을 사용하여 1초마다 이미지 캡쳐
        cv2.imwrite(dirs + "/frame%d.jpg" % count, image)
        #print('Saved frame number :', str(int(cap.get(1))))
        count += 1
        
    #out.write(image)
    
    '''#보기 편하게 이미지를 좌우 반전합니다.
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1)) # 이미지 반전, 0:상하, 1:좌우
    if cv2.waitKey(5) & 0xFF == ord('q'): # q키 누르면 종료
      break'''
    
cap.release()
#out.release()
cv2.destroyAllWindows()

In [21]:
import os
import cv2
import mediapipe as mp
import glob

# 동영상 파일의 경로를 리스트로 가져옴
file_list = glob.glob('./data/*.mp4')

# 미디어 파이프의 손 모델 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# 각 동영상 파일에 대해 처리
for i, file in enumerate(file_list):
    cap = cv2.VideoCapture(file)
    file_name = os.path.splitext(os.path.basename(file))[0]  # 파일 이름 (확장자 제외)

    # 동영상 정보 가져오기
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # FPS
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # 프레임 너비
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 프레임 높이
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # 프레임 수

    # 출력 폴더 생성
    output_dir = f'./data/output_{i}'
    os.makedirs(output_dir, exist_ok=True)

    with mp_hands.Hands(min_detection_confidence=0.5, 
                        min_tracking_confidence=0.5) as hands:
        
        count = 0
        
        while cap.isOpened():
            ret, image = cap.read()

            if not ret:
                break

            elif (int(cap.get(1)) % fps == 0):
                # 손 감지
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = hands.process(image_rgb)

                if results.multi_hand_landmarks:
                    for hand_landmarks in results.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(
                            image,
                            hand_landmarks,
                            mp_hands.HAND_CONNECTIONS,
                            mp_drawing_styles.get_default_hand_landmarks_style(), # 포인트
                            mp_drawing_styles.get_default_hand_connections_style()) # 선


                # 이미지 저장
                image_filename = os.path.join(output_dir, f'frame_{count // fps}.jpg')
                cv2.imwrite(image_filename, image)

            count += 1

    cap.release()

print("모든 동영상 처리가 완료되었습니다.")


모든 동영상 처리가 완료되었습니다.


### 동영상을 프레임 단위로 나누기

In [24]:
# input
filepath = './data/KakaoTalk_20230831_142748198.mp4'
video = cv2.VideoCapture(filepath) #'' 사이에 사용할 비디오 파일의 경로 및 이름을 넣어주도록 함

In [25]:
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)

length : 226
width : 640
height : 480
fps : 24.0


In [26]:
dirs = './data/output2_1'
os.makedirs(dirs)

In [27]:
#프레임을 저장할 디렉토리를 생성
try:
    if not os.path.exists(dirs):
        os.makedirs(dirs)
except OSError:
    print ('Error: Creating directory. ' +  filepath[:-4])

In [28]:
count = 0

while(video.isOpened()):
    ret, image = video.read()
    
    if not ret:
        break
    
    elif (int(video.get(1)) % fps == 0): #앞서 불러온 fps 값을 사용하여 1초마다 이미지 캡쳐
        cv2.imwrite(dirs + "/frame%d.jpg" % count, image)
        print('Saved frame number :', str(int(video.get(1))))
        count += 1
        
video.release()
        

Saved frame number : 24
Saved frame number : 48
Saved frame number : 72
Saved frame number : 96
Saved frame number : 120
Saved frame number : 144
Saved frame number : 168
Saved frame number : 192
Saved frame number : 216
